In [18]:
import pandas as pd
import numpy as np

path = "DB-Output_original.csv"
data = pd.read_csv(path)

data = data.loc[:, :'Initial Solution']
data

,Instance's Name,Initial Solution
0,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
1,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
2,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
3,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
4,c101_21_25.txt,"[['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6..."
...,...,...
35406,c205_21_25.txt,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',..."
35407,c205_21_25.txt,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',..."
35408,c205_21_25.txt,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',..."
35409,c205_21_25.txt,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',..."


In [19]:
init_sol = data['Initial Solution']
print("Type of 'init_sol':", type(init_sol))

init_sol[0]
print("Type of 'init_sol[0]':", type(init_sol[0]))

init_sol.head()

Type of 'init_sol': <class 'pandas.core.series.Series'>
Type of 'init_sol[0]': <class 'str'>


0    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
1    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
2    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
3    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
4    [['D0', 'C21', 'C26', 'D0'], ['D0', 'C65', 'C6...
Name: Initial Solution, dtype: object

Somma delle occorrenza di uno specifico luogo all'interno di tutto il DataFrame, nella colonna "Initial Solution"

In [20]:
print("D0 total: ", init_sol.str.count('D0').sum())
print("C21 total: ", init_sol.str.count('C21').sum())
print("C87 total: ", init_sol.str.count('C87').sum())
print("C27 total: ", init_sol.str.count('C27').sum())

D0 total:  1817188
C21 total:  24067
C87 total:  18864
C27 total:  24666


# Pulizia dei dati

Si prende la colonna contenente i luoghi "Initial Solution" (che è una serie di pandas) e tramite la funzione str.replace() si pulisce la stringa, rimuovendo la punteggiatura speciale (parentesi quadre, virgole e apici) con un carattere null.

Si stampa poi la colonna "Initial Solution" ripulita.

In [21]:
data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
data["Initial Solution"] = data["Initial Solution"].str.replace("]","")
data["Initial Solution"] = data["Initial Solution"].str.replace(",","")
data["Initial Solution"] = data["Initial Solution"].str.replace("'","")

data["Initial Solution"].head()


/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_6366/3585595387.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
/var/folders/_t/_x_rzsrx5vd2_mn90dgnz43h0000gn/T/ipykernel_6366/3585595387.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data["Initial Solution"] = data["Initial Solution"].str.replace("]","")


0    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
1    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
2    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
3    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
4    D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
Name: Initial Solution, dtype: object

# Divisione dei luoghi

Si "isola" la singola colonna "Initial Solution" inserendola nella variabile "init_sol", che diventa una serie a sua volta.

In [22]:
init_sol = data["Initial Solution"]
print(init_sol)
type(init_sol)

0        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
1        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
2        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
3        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
4        D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 D0 C...
                               ...                        
35406    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
35407    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
35408    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
35409    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
35410    D0 C5 C2 C1 C14 C25 C21 D0 D0 C89 C84 C83 C77 ...
Name: Initial Solution, Length: 35411, dtype: object


pandas.core.series.Series

La singola riga della serie equivale ad una stringa. Ad esempio "init_sol[0]" equivale ad una stringa contenente "D0 C21 C26 D0 D0 C65 C61 D0 D0 C24 C50 D0 ..."

Attraverso il metodo split(), inserito in un ciclo for che itera ogni riga di "init_sol", si splitta la stringa in considerazione rispetto al carattere spazio e la si inserisce tramite .append() nella lista "places" inizialmente vuota. Ogni volta che il metodo split() trova uno spazio, viene rilevato un elemento (luogo).

La stringa splittata per singolo elemento viene inserita in una lista chiamata "places", dove sono presenti ancora ripetizioni.

Applicando il cast di tipo da lista a set, e di nuovo da set a lista, si ha in output una lista senza ripetizioni chiamata "distinct_places".
Successivamente, attraverso il metodo .sort(), la lista viene ordinata in modo alfabetico.

In [23]:
places = []
for row in init_sol:
    for elem in row.split():
        places.append(elem)

distinct_places = list(set(places))

distinct_places.sort()
print("\nLa lista di tutti i diversi luoghi (ordinati) è:\n", distinct_places)

print("\nLa lista contiene", len(distinct_places), "diversi luoghi")
print("\nPrint di diversi luoghi:\n", distinct_places[0], distinct_places[5], distinct_places[23])



La lista di tutti i diversi luoghi (ordinati) è:
 ['C1', 'C10', 'C100', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C3', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C4', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C5', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C6', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C7', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C8', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C9', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'D0', 'S0', 'S1', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S2', 'S20', 'S3', 'S4', 'S5', 'S7', 'S9']

La lista contiene 120 diversi luoghi

Print di diversi luoghi:
 C1 C13 C3


# Creazione della matrice 

Si procede quindi alla creazione della matrice che ha come dimensioni la lunghezza della lista "distinct_places", ovvero il numero dei diversi luoghi presenti nel dataframe iniziale 

In [47]:
print("\nLa lista 'distinct_places' contiene", len(distinct_places), "diversi luoghi\n")

pd.set_option('display.max_columns', None)
matrix = pd.DataFrame(0, index=distinct_places, columns=distinct_places)
matrix.head(105)

# usiamo un dataframe per avere indici secondo i quali andare ad inserire le corrispondenze, 
# successivamente si converte in matrice numpy


La lista 'distinct_places' contiene 120 diversi luoghi



,C1,C10,C100,C11,C12,C13,C14,C15,C16,C17,C18,C19,C2,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C3,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C4,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C5,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C6,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C7,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C8,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C9,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,D0,S0,S1,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S2,S20,S3,S4,S5,S7,S9
C1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
S1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
S10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# metodo 1
index = 0            # Python's indexing starts at zero
for item in places:   # Python's for loops are a "for each" loop 
    print(index, item)
    index += 1
    
    
# metodo 2  
for i, j in enumerate(places):
    print(i, j)
    

for index, item in enumerate(items, start=0):   # default is zero
    print(index, item)

In [40]:
places_matrix = np.zeros((a, a))
print("Matrice dei vari luoghi di dimensione", places_matrix.shape, ":\n", places_matrix)

Matrice dei vari luoghi di dimensione (120, 120) :
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
place_ridotto = places[:119]

In [59]:
for index, item in enumerate(place_ridotto, start=0):   # default is zero
    print(index, item)

0 D0
1 C21
2 C26
3 D0
4 D0
5 C65
6 C61
7 D0
8 D0
9 C24
10 C50
11 D0
12 D0
13 C25
14 D0
15 D0
16 C6
17 C91
18 D0
19 D0
20 C9
21 D0
22 D0
23 C2
24 D0
25 D0
26 C90
27 C85
28 D0
29 D0
30 C32
31 S11
32 D0
33 D0
34 C82
35 C81
36 S20
37 D0
38 D0
39 C18
40 D0
41 D0
42 C15
43 S7
44 D0
45 D0
46 C100
47 S3
48 D0
49 D0
50 C16
51 S7
52 D0
53 D0
54 C97
55 S3
56 D0
57 D0
58 C39
59 S12
60 D0
61 D0
62 C94
63 S4
64 D0
65 D0
66 C58
67 S16
68 D0
69 D0
70 C73
71 S20
72 D0
73 D0
74 C21
75 C26
76 D0
77 D0
78 C65
79 C61
80 D0
81 D0
82 C24
83 C50
84 D0
85 D0
86 C25
87 D0
88 D0
89 C6
90 C91
91 D0
92 D0
93 C9
94 D0
95 D0
96 C2
97 D0
98 D0
99 C90
100 C85
101 D0
102 D0
103 C32
104 S11
105 D0
106 D0
107 C82
108 C81
109 S20
110 D0
111 D0
112 C18
113 D0
114 D0
115 C15
116 S7
117 D0
118 D0


In [68]:
for index, item in enumerate(place_ridotto):
    index_a = index
    index_b = index+1
    
    #item_a = place_red[index]
    #item_b = place_red[index+1]
    
    print(index_a, index_b)
    
    places_matrix[index_a][index_b] = 1
    

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 46
46 47
47 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 56
56 57
57 58
58 59
59 60
60 61
61 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 93
93 94
94 95
95 96
96 97
97 98
98 99
99 100
100 101
101 102
102 103
103 104
104 105
105 106
106 107
107 108
108 109
109 110
110 111
111 112
112 113
113 114
114 115
115 116
116 117
117 118
118 119


In [69]:
places_matrix

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
print(places[:30])
print(places.index('C21'))
len(places)

['D0', 'C21', 'C26', 'D0', 'D0', 'C65', 'C61', 'D0', 'D0', 'C24', 'C50', 'D0', 'D0', 'C25', 'D0', 'D0', 'C6', 'C91', 'D0', 'D0', 'C9', 'D0', 'D0', 'C2', 'D0', 'D0', 'C90', 'C85', 'D0', 'D0']
1


4522477

In [70]:
verifica = pd.DataFrame(places_matrix, index=distinct_places, columns=distinct_places)
verifica

,C1,C10,C100,C11,C12,C13,C14,C15,C16,C17,C18,C19,C2,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C3,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C4,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C5,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C6,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C7,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C8,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C9,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,D0,S0,S1,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S2,S20,S3,S4,S5,S7,S9
C1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C100,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C12,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
S4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.